In [ ]:
# Modified Resnet

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=0.5):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=0.5, beta_weight=0.5):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

In [ ]:
# using hybrid sampling

In [6]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import Counter
from torch.utils.data import Subset
from torch.utils.data import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter
import numpy as np
import random

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L') 
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([  
    transforms.Resize((64, 64)),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

def undersample_class(dataset, majority_class, target_count):
    majority_indices = [i for i, (_, label) in enumerate(dataset) if label == majority_class]
    undersampled_majority_indices = np.random.choice(majority_indices, size=target_count, replace=False)
    return undersampled_majority_indices

class OversampleDataset(Dataset):
    def __init__(self, dataset, labels, minority_classes):
        self.dataset = dataset
        self.labels = labels
        self.minority_classes = minority_classes
        
        self.minority_indices = [i for i, label in enumerate(self.labels) if label in self.minority_classes]
        self.majority_indices = [i for i, label in enumerate(self.labels) if label not in self.minority_classes]
        
        self.oversampled_minority_indices = random.choices(self.minority_indices, k=min(len(self.majority_indices), len(self.minority_indices) * 5))
        
        self.indices = self.majority_indices + self.oversampled_minority_indices
    
    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx] % len(self.dataset)
        return self.dataset[actual_idx]

def hybrid_sampling(dataset, labels, majority_class, minority_classes):
    class_counts = Counter(labels)
    
    target_count = 500
    undersampled_majority_indices = undersample_class(dataset, majority_class, target_count)
    
    minority_indices = [i for i, (_, label) in enumerate(dataset) if label in minority_classes]
    
    combined_indices = list(undersampled_majority_indices) + minority_indices
    
    return Subset(dataset, combined_indices)

data_dir = 'D:/project/with_roi_histactual2'
dataset = CustomDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in dataset]  
class_counts = Counter(labels)

print("Class counts:", class_counts)

majority_class = max(class_counts, key=class_counts.get)
minority_classes = [cls for cls in class_counts if cls != majority_class]

print("Majority class:", majority_class)
print("Minority classes:", minority_classes)

train_size = int(0.85 * len(dataset))  
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataset = hybrid_sampling(train_dataset, labels, majority_class, minority_classes)

train_dataset = OversampleDataset(train_dataset, labels, minority_classes)



train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=32, 
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=32, 
                          shuffle=False)

print(f"Full dataset size: {len(dataset)}")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
ccriterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5)


def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device) 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) 
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    
    train_accuracy = calculate_accuracy(train_loader, model)
    test_accuracy = calculate_accuracy(test_loader, model)
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_sli

KeyboardInterrupt: 

In [ ]:
#using SMOTE

In [10]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from imblearn.over_sampling import SMOTE

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

features = []
labels = []
for img, label in dataset:
    features.append(img.numpy().flatten())  # Flatten each image to a 1D array
    labels.append(label)

# Apply SMOTE to oversample minority classes
smote = SMOTE()
features_resampled, labels_resampled = smote.fit_resample(features, labels)
print(f"Class distribution after SMOTE: {Counter(labels_resampled)}")

# Convert resampled features and labels back to tensors
features_resampled = np.array(features_resampled).reshape(-1, 1, 64, 64)
features_tensor = torch.tensor(features_resampled, dtype=torch.float32)
labels_tensor = torch.tensor(labels_resampled, dtype=torch.long)

# Create a new dataset and split it
resampled_dataset = TensorDataset(features_tensor, labels_tensor)
train_size = int(0.85 * len(resampled_dataset))
test_size = len(resampled_dataset) - train_size
train_dataset, test_dataset = random_split(resampled_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

d:\pc_softwares\Python\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/100], Loss: 1.9715, Train Accuracy: 66.67%, Test Accuracy: 64.86%
Epoch [2/100], Loss: 0.9772, Train Accuracy: 79.15%, Test Accuracy: 76.29%
Epoch [3/100], Loss: 0.6829, Train Accuracy: 84.16%, Test Accuracy: 79.90%
Epoch [4/100], Loss: 0.5606, Train Accuracy: 87.71%, Test Accuracy: 80.87%
Epoch [5/100], Loss: 0.4913, Train Accuracy: 89.63%, Test Accuracy: 82.31%
Epoch [6/100], Loss: 0.4221, Train Accuracy: 89.94%, Test Accuracy: 83.87%
Epoch [7/100], Loss: 0.3846, Train Accuracy: 93.35%, Test Accuracy: 85.80%
Epoch [8/100], Loss: 0.3408, Train Accuracy: 94.54%, Test Accuracy: 86.16%
Epoch [9/100], Loss: 0.3290, Train Accuracy: 93.35%, Test Accuracy: 86.76%
Epoch [10/100], Loss: 0.3034, Train Accuracy: 94.47%, Test Accuracy: 86.64%
Epoch [11/100], Loss: 0.2763, Train Accuracy: 93.66%, Test Accuracy: 85.92%
Epoch [12/100], Loss: 0.2720, Train Accuracy: 96.13%, Test Accuracy: 86.40%
Epoch [13/100], Loss: 0.2481, Train Accuracy: 96.22%, Test Accuracy: 87.36%
Epoch [14/100], Loss:

In [ ]:
# weight balancing with Inverse Number of Samples (INS) weights

In [12]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
# Initialize model, criterion, optimizer, and scheduler
class_counts = torch.zeros(3).to(device)  # Assuming 3 classes
for _, label in train_dataset:
    class_counts[label] += 1

# Calculate Inverse Number of Samples (INS) weights
weights = 1.0 / class_counts
weights = weights / weights.sum()  # Normalize weights

# Define the criterion with INS weights
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

d:\pc_softwares\Python\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/100], Loss: 2.6976, Train Accuracy: 17.79%, Test Accuracy: 20.64%
Epoch [2/100], Loss: 1.6858, Train Accuracy: 49.79%, Test Accuracy: 44.77%
Epoch [3/100], Loss: 1.3512, Train Accuracy: 53.43%, Test Accuracy: 41.02%
Epoch [4/100], Loss: 1.1810, Train Accuracy: 75.86%, Test Accuracy: 62.47%
Epoch [5/100], Loss: 1.0043, Train Accuracy: 64.17%, Test Accuracy: 44.24%
Epoch [6/100], Loss: 0.8546, Train Accuracy: 73.64%, Test Accuracy: 51.47%
Epoch [7/100], Loss: 0.7815, Train Accuracy: 82.16%, Test Accuracy: 57.10%
Epoch [8/100], Loss: 0.6747, Train Accuracy: 82.11%, Test Accuracy: 55.76%
Epoch [9/100], Loss: 0.5580, Train Accuracy: 84.34%, Test Accuracy: 56.84%
Epoch [10/100], Loss: 0.4737, Train Accuracy: 80.93%, Test Accuracy: 52.55%
Epoch [11/100], Loss: 0.3805, Train Accuracy: 91.95%, Test Accuracy: 58.18%
Epoch [12/100], Loss: 0.3685, Train Accuracy: 89.54%, Test Accuracy: 60.05%
Epoch [13/100], Loss: 0.3592, Train Accuracy: 94.23%, Test Accuracy: 60.86%
Epoch [14/100], Loss:

In [ ]:
# weight balancing with Inverse of Square Root of Number of Samples (ISNS) weights

In [13]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)

# Initialize model, criterion, optimizer, and scheduler
class_counts = torch.zeros(3)  # Assuming 3 classes
label_counts = Counter([label for _, label in train_dataset])
num_classes = len(label_counts)
class_weights = []

# Calculate weights based on the Inverse of Square Root of Number of Samples (ISNS)
for label in range(num_classes):
    if label_counts[label] > 0:
        class_weights.append(1.0 / np.sqrt(label_counts[label]))
    else:
        class_weights.append(0.0)  # Assign 0 weight if no samples in class

# Normalize weights to ensure they sum up to 1
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
class_weights = class_weights / class_weights.sum()

# Define the criterion with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

In [ ]:
# weight balancing with Effective Number of Samples (ENS) weights

In [14]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def calculate_class_weights(labels, beta=0.99):
    """Calculates class weights using Effective Number of Samples (ENS) approach."""
    classes, counts = np.unique(labels, return_counts=True)
    effective_num = 1.0 - np.power(beta, counts)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * len(classes)  # Normalize weights
    return torch.tensor(weights, dtype=torch.float32).to(device)

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model

train_labels = [label for _, label in train_dataset] 

class_weights = calculate_class_weights(train_labels)

model = ModifiedResNet(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")

Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0

In [ ]:
# offline data augmentation

In [15]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def apply_offline_augmentation(source_dir, dest_dir, num_augmentations=5):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    transform_augment = transforms.Compose([
        transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5)),  # Brightness & Contrast
        transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.5),  # Sharpness
        transforms.RandomHorizontalFlip(p=0.5),  # Flip
        transforms.RandomRotation(degrees=(-20, 20)),  # Rotation
        transforms.RandomResizedCrop(64, scale=(0.8, 1.2)),  # Zoom
    ])
    
    for label in os.listdir(source_dir):
        label_dir = os.path.join(source_dir, label)
        dest_label_dir = os.path.join(dest_dir, label)
        
        if not os.path.exists(dest_label_dir):
            os.makedirs(dest_label_dir)
        
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            image = Image.open(img_path).convert('L')
            
            # Save original image to the new directory
            image.save(os.path.join(dest_label_dir, img_name))
            
            # Apply augmentations and save
            for i in range(num_augmentations):
                augmented_image = transform_augment(image)
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.png"
                augmented_image.save(os.path.join(dest_label_dir, aug_img_name))

# Apply offline augmentation
source_dir = 'D:/project/with_roi_histactual2'
augmented_dir = 'D:/project/augmented_data1'
apply_offline_augmentation(source_dir, augmented_dir)

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load dataset
full_dataset = CustomDataset(root_dir=augmented_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cuda
Checking directory: D:/project/augmented_data1\0
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi_aug_0.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi_aug_1.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi_aug_2.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi_aug_3.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice0_L_roi_aug_4.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/augmented_data1\0\329637_series8_slice1_L_roi_aug_0.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice1_L_roi_aug_1.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice1_L_roi_aug_2.png
Loaded image: D:/project/augmented_data1\0\329637_series8_slice1_L_roi_aug_3.png
Loaded image: D:/project/augmented_data1\0\329637_ser

In [ ]:
# online data augmentation

In [22]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in sorted(os.listdir(root_dir)):
            label_dir = os.path.join(root_dir, label)
            print(f"Checking directory: {label_dir}")  
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    if img_name.endswith('.jpg') or img_name.endswith('.png'): 
                        img_path = os.path.join(label_dir, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(int(label)) 
                        print(f"Loaded image: {img_path}")  

        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure grayscale image loading
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transformations to apply to the images
transform = transforms.Compose([
        transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5)),  # Brightness & Contrast
        transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.5),  # Sharpness
        transforms.RandomHorizontalFlip(p=0.5),  # Flip
        transforms.RandomRotation(degrees=(-20, 20)),  # Rotation
        transforms.RandomResizedCrop(64, scale=(0.8, 1.2)),  # Zoom
        transforms.ToTensor(),  # Convert PIL image to tensor
])

# Load dataset
data_dir = 'D:/project/with_roi_histactual2'
full_dataset = CustomDataset(root_dir=data_dir, transform=transform)

# Split dataset into training (85%) and test (15%) sets
train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the modified CNN model with Batch Normalization and Weight Initialization
class ModifiedResNetIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_weight=1.0):
        super(ModifiedResNetIdentityBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.skip_weight = skip_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.conv(x)))
        out = out+self.skip_weight * identity
        return self.relu(out)

class ModifiedResNetMultipleIdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, lambda_weight=1.0, beta_weight=1.0):
        super(ModifiedResNetMultipleIdentityBlock, self).__init__()
        self.identity_block1 = ModifiedResNetIdentityBlock(in_channels, out_channels, skip_weight=lambda_weight)
        self.identity_block2 = ModifiedResNetIdentityBlock(out_channels, out_channels, skip_weight=lambda_weight)
        self.skip_weight = beta_weight
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = x
        out = self.identity_block1(x)
        out = self.identity_block2(out)
        out = out+self.skip_weight * identity
        return out

class ModifiedResNetConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2):
        super(ModifiedResNetConvolutionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=1, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels1)
        self.conv2 = nn.Conv2d(out_channels1, out_channels1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels1)
        self.conv3 = nn.Conv2d(out_channels1, out_channels2, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels2)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv2d(in_channels, out_channels2, kernel_size=1, stride=2, bias=False)
        self._initialize_weights() 

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out+identity
        return self.relu(out)

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1, stride=2, bias=False)
        self.skip_conv2 = nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False)
        self.skip_conv3 = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Stage definitions based on specified architecture
        self.stage1_conv = nn.Conv2d(1, 16, kernel_size=1, stride=1, bias=False)
        self.stage1_bn = nn.BatchNorm2d(16)
        
        # Defining stages according to block specifications
        self.stage3 = self._make_stage(16, 16, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage4 = ModifiedResNetConvolutionBlock(16, 16, 32)
        self.stage5 = self._make_stage(32, 32, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)
        self.stage6 = ModifiedResNetConvolutionBlock(32, 32, 64)
        self.stage7 = self._make_stage(64, 64, ModifiedResNetMultipleIdentityBlock, num_blocks=2, lambda_weight=0.5, beta_weight=0.5)

        # Final layers
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1024, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights using Xavier initialization for layers with weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def _make_stage(self, in_channels, out_channels, block, num_blocks, **kwargs):
        layers = [block(in_channels, out_channels, **kwargs)]
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.relu(self.bn(self.stage1_conv(x)))
        x = self.maxpool(x)
        sc1 = x
        sc2 = x
        x = self.stage3(x)
        x = self.stage4(x)
        sc3 = x
        x = self.stage5(x)
        sco2 = self.skip_conv2(sc2)
        sco2 = F.interpolate(sco2, size=x.shape[2:], mode='bilinear', align_corners=False)  
        x = x+sco2
        x = self.stage6(x)
        x = self.stage7(x)
        sco1 = self.skip_conv1(sc1)
        sco3 = self.skip_conv3(sc3)
        sco1 = F.interpolate(sco1, size=x.shape[2:], mode='bilinear', align_corners=False)
        sco3 = F.interpolate(sco3, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = x+sco3
        x = x+sco1
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and test the model
model = ModifiedResNet(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=5, verbose=True)

# Function to calculate accuracy with device support
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model with accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate accuracy for training and test sets
    train_accuracy = calculate_accuracy(train_loader, model, device)
    test_accuracy = calculate_accuracy(test_loader, model, device)
    
    # Update scheduler
    scheduler.step(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

print("Training complete.")


Using device: cuda
Checking directory: D:/project/with_roi_histactual2\0
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice0_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice1_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\329637_series8_slice2_L_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice2_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice3_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice4_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\390116_series9_slice5_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice0_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0\406320_series9_slice1_R_roi.jpg
Loaded image: D:/project/with_roi_histactual2\0